# 1. Data Preprocess

## 1.1 Data Collection 

Download the datasets from [Cornell Movie Datasets Website](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) and unzip the data into txt files.



In [1]:
! wget -nc "http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip"
! unzip cornell_movie_dialogs_corpus.zip
! rm cornell_movie_dialogs_corpus.zip

--2022-04-19 19:23:37--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  58.6MB/s    in 0.2s    

2022-04-19 19:23:37 (58.6 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-

## 1.2 Data Cleaning & Wrangling

Clean the data and convert it into the form of a dialog.

In [1]:
# open dialog files
movie_lines = open('cornell movie-dialogs corpus/movie_lines.txt', encoding='utf-8',errors='ignore').read().split('\n')
movie_conversations = open('cornell movie-dialogs corpus/movie_conversations.txt', encoding='utf-8',errors='ignore').read().split('\n')

In [2]:
# build a dictionary to record (line_number, dialog) mappings
line_to_dialog = {}
for line in movie_lines:
  line_splited = line.split(' +++$+++ ')
  line_to_dialog[line_splited[0]] = line_splited[-1]

In [3]:
# build dialog fragments
dialog_fragments = []
for conversation in movie_conversations:
  ## convert u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197'] to 'L194', 'L195', 'L196', 'L197'
  dialog_instance = conversation.split(' +++$+++ ')[-1][1:-1]
  ## convert 'L194', 'L195', 'L196', 'L197' to ['L194', 'L195', 'L196', 'L197']
  dialog_fragments.append(dialog_instance.replace("'", " ").replace(",","").split())

In [4]:
# convert dialog fragments into (question, answer) pairs
questions = []
answers = []

for frag in dialog_fragments:
    for i in range(1, len(frag)):
        questions.append(line_to_dialog[frag[i-1]])
        answers.append(line_to_dialog[frag[i]])

In [5]:
# show questions and answers pairs
def qa_show(num):
  for i in range(num):
    print('-------------------------------------------------\n')
    print(f"Dialog A: {questions[i]}\n")
    print(f"Dialog B: {answers[i]}\n")

qa_show(5)

-------------------------------------------------

Dialog A: Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.

Dialog B: Well, I thought we'd start with pronunciation, if that's okay with you.

-------------------------------------------------

Dialog A: Well, I thought we'd start with pronunciation, if that's okay with you.

Dialog B: Not the hacking and gagging and spitting part.  Please.

-------------------------------------------------

Dialog A: Not the hacking and gagging and spitting part.  Please.

Dialog B: Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

-------------------------------------------------

Dialog A: You're asking me out.  That's so cute. What's your name again?

Dialog B: Forget it.

-------------------------------------------------

Dialog A: No, no, it's my fault -- we didn't have a proper introduction ---

Dialog B: Cameron.



## 1.3 Text Preprocessing

Preprocess the texts.


In [6]:
# Global variables
TEXT_LIMIT = 13

In [7]:
# filter out long dialogs
def filter_long_texts(questions, answers, limit):
    short_questions = []
    short_answers = []
    for i in range(len(questions)):
        # if len(questions[i]) <= TEXT_LIMIT and len(answers[i]) <=TEXT_LIMIT:
        if len(questions[i].split()) <= TEXT_LIMIT:
            short_questions.append(questions[i])
            short_answers.append(answers[i])
    return short_questions, short_answers

In [8]:
filtered_questions, filtered_answers = filter_long_texts(questions, answers, TEXT_LIMIT)

In [9]:
# clean the texts
import re

replacement_patterns = [
  (r'won\'t', 'will not'),
  (r'can\'t', 'cannot'),
  (r'i\'m', 'i am'),
  (r'ain\'t', 'is not'),
  (r'(\w+)\'ll', '\g<1> will'),
  (r'(\w+)n\'t', '\g<1> not'),
  (r'(\w+)\'ve', '\g<1> have'),
  (r'(\w+)\'s', '\g<1> is'),
  (r'(\w+)\'re', '\g<1> are'),
  (r'(\w+)\'d', '\g<1> would'),
]

class TextCleaner(object):
  def __init__(self, patterns=replacement_patterns):
    self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    
  def replace(self, text):
    s = text
    for (pattern, replace) in self.patterns:
      s = re.sub(pattern, replace, s)
    return s

cleaner = TextCleaner()

# Function for preprocessing the given text
def preprocess_text(text):
    
    # Lowercase the text
    text = text.lower()
    
    # Decontracting the text (e.g. it's -> it is)
    text = cleaner.replace(text)
    
    # Remove the punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    # Remove extra spaces
    text = re.sub(r"[ ]+", " ", text)
    
    return text

In [10]:
cleaned_questions = [preprocess_text(que) for que in filtered_questions]
cleaned_answers = [' '.join(ans.split()[:TEXT_LIMIT-2]) for ans in filtered_answers]

del(questions, answers, filtered_questions, filtered_answers)

for i in range(5):
    print('-------------------------------------------------\n')
    print(f'Dialog A: {cleaned_questions[i]}\n')
    print(f'Dialog B: {cleaned_answers[i]}\n')

-------------------------------------------------

Dialog A: well i thought we would start with pronunciation if that is okay with you 

Dialog B: Not the hacking and gagging and spitting part. Please.

-------------------------------------------------

Dialog A: not the hacking and gagging and spitting part please 

Dialog B: Okay... then how 'bout we try out some French cuisine. Saturday?

-------------------------------------------------

Dialog A: you are asking me out that is so cute what is your name again 

Dialog B: Forget it.

-------------------------------------------------

Dialog A: no no it is my fault we did not have a proper introduction 

Dialog B: Cameron.

-------------------------------------------------

Dialog A: cameron 

Dialog B: The thing is, Cameron -- I'm at the mercy of a



After preprocessing the dataset, we should add a start tag (e.g. `<start>`) and an end tag (e.g. `<end>`) to answers. Remember that we will only add these tags to answers and not questions.

When Seq2Seq model is generating the word answers, we can first send it the `<start>` to begin the word generation. When `<end>` is generated, we will stop the iteration.


In [11]:
cleaned_answers = ["starttoken " + ans + " endtoken" for ans in cleaned_answers]

In [12]:
# trim the data in case of running out of RAM
cleaned_questions = cleaned_questions[:30000]
cleaned_answers = cleaned_answers[:30000]

## 1.4 Input Encoding

Convet String input into numerical values


In [13]:
import numpy as np
from tensorflow.keras import preprocessing, utils
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
NUM_WORDS = 3500

In [15]:
# Initialize the tokenizer
tokenizer = preprocessing.text.Tokenizer(num_words = NUM_WORDS, oov_token='nulltoken')

# Fit the tokenizer to questions and answers
tokenizer.fit_on_texts(cleaned_questions + cleaned_answers)

# Get the total vocab size
VOCAB_SIZE = len(tokenizer.word_index) + 1
# display(tokenizer.word_index)
print( 'VOCAB SIZE : {}'.format(VOCAB_SIZE))

VOCAB SIZE : 14987


In [16]:
tokenizer.word_index["starttoken"]

2

In [17]:
### encoder input data

# Tokenize the questions
tokenized_questions = tokenizer.texts_to_sequences(cleaned_questions)

# Get the length of longest sequence
# maxlen_questions = max([len(x) for x in tokenized_questions])

# Pad the sequences
padded_questions = pad_sequences(tokenized_questions, maxlen=TEXT_LIMIT, padding='post')

# Convert the sequences into array
encoder_input_data = np.array(padded_questions)
# print(encoder_input_data[0])
print(encoder_input_data.shape, TEXT_LIMIT)

(30000, 13) 13


In [18]:
### decoder input data

# Tokenize the answers
tokenized_answers = tokenizer.texts_to_sequences(cleaned_answers)

# Get the length of longest sequence
# maxlen_answers = max([len(x) for x in tokenized_answers])

# Pad the sequences
padded_answers = pad_sequences(tokenized_answers, maxlen=TEXT_LIMIT, padding='post')

# Convert the sequences into array
decoder_input_data = np.array(padded_answers)

print(decoder_input_data.shape, TEXT_LIMIT)

(30000, 13) 13


In [19]:
### decoder_output_data

# Iterate through index of tokenized answers
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]

# Pad the tokenized answers
padded_answers = pad_sequences(tokenized_answers, maxlen = TEXT_LIMIT, padding = 'post')

# One hot encode
onehot_answers = utils.to_categorical(padded_answers, NUM_WORDS+1)

In [20]:
# Convert to numpy array
decoder_output_data = np.array(onehot_answers)
# print(decoder_output_data[0])
print(decoder_output_data.shape)

del (onehot_answers)

(30000, 13, 3501)


In [21]:
# Import the libraries
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import softmax
from tensorflow.keras.callbacks import ModelCheckpoint

In [22]:
print("Num GPUs Available: ", tf.config.experimental_list_devices())

Num GPUs Available:  ['/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:XLA_CPU:0', '/job:localhost/replica:0/task:0/device:XLA_GPU:0', '/job:localhost/replica:0/task:0/device:GPU:0']


In [23]:
# Hyper parameters
BATCH_SIZE = 32
EPOCHS = 200
VOCAB_SIZE = NUM_WORDS + 1

In [24]:
### Encoder Input
embed_dim = 50
num_lstm = 400

# Input for encoder
encoder_inputs = Input(shape = (None, ), name='encoder_inputs')

# Embedding layer
# Why mask_zero = True? https://www.tensorflow.org/guide/keras/masking_and_padding
encoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = embed_dim, mask_zero = True, name='encoder_embedding')(encoder_inputs)

# LSTM layer (that returns states in addition to output)
encoder_outputs, state_h, state_c = LSTM(units = num_lstm, return_state = True, name='encoder_lstm')(encoder_embedding)

# Get the states for encoder
encoder_states = [state_h, state_c]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
### Decoder

# Input for decoder
decoder_inputs = Input(shape = (None,  ), name='decoder_inputs')

# Embedding layer
decoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = embed_dim , mask_zero = True, name='decoder_embedding')(decoder_inputs)

# LSTM layer (that returns states and sequences as well)
decoder_lstm = LSTM(units = num_lstm , return_state = True , return_sequences = True, name='decoder_lstm')

# Get the output of LSTM layer, using the initial states from the encoder
decoder_outputs, _, _ = decoder_lstm(inputs = decoder_embedding, initial_state = encoder_states)

# Dense layer
decoder_dense = Dense(units = VOCAB_SIZE, activation = softmax, name='decoder_outputs') 

# Get the output of Dense layer
output = decoder_dense(decoder_outputs)

In [26]:
# Create the model
model = Model([encoder_inputs, decoder_inputs], output)

In [27]:
# Compile the model
model.compile(optimizer='adam', metrics=['acc'], loss = "categorical_crossentropy")

In [28]:
# Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, None, 50)     175050      encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, None, 50)     175050      decoder_inputs[0][0]             
______________________________________________________________________________________________

In [29]:
# Train the model
model.fit(
    x = {"encoder_inputs": encoder_input_data, "decoder_inputs": decoder_input_data}, 
    y = {'decoder_outputs': decoder_output_data}, 
    batch_size = BATCH_SIZE, 
    epochs = EPOCHS
) 



[2022-04-19 20:41:44.799 ip-172-16-3-167:4839 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-19 20:41:44.825 ip-172-16-3-167:4839 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Train on 30000 samples
Epoch 1/200
30000/30000 [==============================] - 42s 1ms/sample - loss: 3.2326 - acc: 0.2397
Epoch 2/200
30000/30000 [==============================] - 41s 1ms/sample - loss: 2.9283 - acc: 0.2764
Epoch 3/200
30000/30000 [==============================] - 40s 1ms/sample - loss: 2.7753 - acc: 0.2957
Epoch 4/200
30000/30000 [==============================] - 41s 1ms/sample - loss: 2.6689 - acc: 0.3066
Epoch 5/200
30000/30000 [==============================] - 40s 1ms/sample - loss: 2.5826 - acc: 0.3147
Epoch 6/200
30000/30000 [==============================] - 41s 1ms/sample - loss: 2.5071 - acc: 0.3214
Epoch 7/200
30000/30000 [==============================] - 40s 1ms/sample - loss: 2.43

In [30]:
model.save(filepath=f"weight_{EPOCHS}.h5")

# Inference

Build the inference model(the same as training model) and load the trained weight. 

In [31]:
# Load the final model
model.load_weights('weight_200.h5') 
print("Model Weight Loaded!")

Model Weight Loaded!


In [32]:
# Function for making inference
def make_inference_models():
    
    # Create a model that takes encoder's input and outputs the states for encoder
    encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)
    
    # Create two inputs for decoder which are hidden state (or state h) and cell state (or state c)
    decoder_state_input_h = Input(shape = (num_lstm, ), name='decoder_state_input_h')
    decoder_state_input_c = Input(shape = (num_lstm, ), name='decoder_state_input_c')
    
    # Store the two inputs for decoder inside a list
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    # Pass the inputs through LSTM layer you have created before
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state = decoder_states_inputs)
    
    # Store the outputted hidden state and cell state from LSTM inside a list
    decoder_states = [state_h, state_c]

    # Pass the output from LSTM layer through the dense layer you have created before
    decoder_outputs = decoder_dense(decoder_outputs)

    # Create a model that takes decoder_inputs and decoder_states_inputs as inputs and outputs decoder_outputs and decoder_states
    decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                          [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [33]:
# Function for converting strings to tokens
def str_to_tokens(sentence: str, tokenizer, maxlen_questions=TEXT_LIMIT):

    # Lowercase the sentence and split it into words
    words = sentence.lower().split()

    tokens_list = tokenizer.texts_to_sequences([sentence])

    # Pad the sequences to be the same length
    return pad_sequences(tokens_list , maxlen = maxlen_questions, padding = 'post')

In [34]:
# Initialize the model for inference
enc_model , dec_model = make_inference_models()

In [38]:
# Iterate through the number of times you want to ask question
try:
    for _ in range(5):

        # Get the input and predict it with the encoder model
        encoder_inputs = str_to_tokens(preprocess_text(input('Enter question : ')), tokenizer)
        states_values = enc_model.predict(encoder_inputs)

        # Initialize the decoder input sequence with the starttoken index
        # Reshape this to be a (1, 1) array, since it is a sequence of 1 sample for 1 timestep
        empty_target_seq = np.zeros((1, 1))

        # Update the target sequence with index of "start"
        empty_target_seq[0, 0] = tokenizer.word_index["starttoken"]
        # Initialize the stop condition with False
        stop_condition = False

        # Initialize the decoded words with an empty string
        decoded_translation = []

        # While stop_condition is false
        while not stop_condition :

            # Predict the (target sequence + the output from encoder model) with decoder model
            dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
            # Get the index for sampled word using the dec_outputs
            # dec_outputs is a numpy array of the shape (sample, timesteps, VOCAB_SIZE)
            # To start, we can just pick the word with the higest probability - greedy search
            sampled_word_index = np.argmax(dec_outputs[0, -1, :])
            
            # Initialize the sampled word with None
            sampled_word = None

            # Iterate through words and their indexes
            for word, index in tokenizer.word_index.items() :

                # If the index is equal to sampled word's index
                if sampled_word_index == index :

                    # Add the word to the decoded string
                    decoded_translation.append(word)

                    # Update the sampled word
                    sampled_word = word

            # If sampled word is equal to "end" OR the length of decoded string is more that what is allowed
            if sampled_word == 'endtoken' or len(decoded_translation) > TEXT_LIMIT:

                # Make the stop_condition to true
                stop_condition = True

            # Initialize back the target sequence to zero - array([[0.]])    
            empty_target_seq = np.zeros(shape = (1, 1))  

            # Update the target sequence with index of "start"
            empty_target_seq[0, 0] = sampled_word_index

            # Get the state values
            states_values = [h, c] 

            # Print the decoded string
        print(' '.join(decoded_translation[:-1]))
except KeyboardInterrupt:
    print('Ending conversational agent')

Enter question :  how are you


been better i read about what's been happening with you i


Enter question :  how are you doing


not too good


Enter question :  what is your name


nulltoken nulltoken and at your service sir


Enter question :  what is the weather today


what


Enter question :  what are you saying


nulltoken another guy came in asking me for a while
